In [1]:
#standard libraries
import pandas as pd
import numpy as np

#need this to get data from my googlesheet
from io import StringIO
import matplotlib.pyplot as plt
import requests

#sklearn library is large
#notice how the import is specifying class and funciton
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Construct the export URL
url = 'https://docs.google.com/spreadsheets/d/1KRNYs4BsOV7IplHCNAxHfj3jdDdmbLMCkFIbM6E3yuM/export?format=csv&gid=1434391104'

# Fetch the CSV data
response = requests.get(url)
data = response.content

# Read the CSV data into a DataFrame
df = pd.read_csv(StringIO(data.decode('utf-8')))

In [3]:
print(df.dtypes)

Property Name                    object
Is Carriage House                  bool
Bedrooms                          int64
Occupancy                         int64
Miles to Beach                  float64
VRBO Rating                     float64
Zestimate at time of listing     object
Sale Price                       object
Year                              int64
dtype: object


In [4]:
#its a little easier for a step later on if I remove all the spaces and such from the column names
df = df.rename(columns={
    'Property Name': 'Property_Name',
    'Is Carriage House':  'Is_Carriage_House',
    'Bedrooms': 'Bedrooms',
    'Occupancy': 'Occupancy',
    'Miles to Beach': 'Miles_to_Beach',
    'VRBO Rating' : 'VRBO_Rating',
    'Zestimate at time of listing': 'Zestimate_at_time_of_listing',
    'Sale Price': 'Sale_Price',
    'Year': 'Year'
})
df

,Property_Name,Is_Carriage_House,Bedrooms,Occupancy,Miles_to_Beach,VRBO_Rating,Zestimate_at_time_of_listing,Sale_Price,Year
0,114 North Winston Villa,False,5,14,0.10,1.00,"2,974,000","3,043,000",2023
1,151 Coopersmith Cottage & Carriage House,True,5,12,3.12,0.88,"1,554,000","1,453,000",2024
2,201 Rosemary Beach Inn,True,1,2,2.92,0.93,"2,386,000","2,433,000",2024
3,202 Rosemary Beach Inn,True,1,2,1.93,0.86,"4,357,000","4,339,000",2024
4,203 Rosemary Beach Inn,True,1,2,4.30,0.97,"1,860,000","1,926,000",2024
...,...,...,...,...,...,...,...,...,...
227,White Dove,False,5,14,1.66,0.96,"3,033,000","3,003,000",2023
228,Wiggle Room,False,2,4,1.04,0.87,"5,677,000","5,648,000",2023
229,Winston Manor,False,6,14,0.23,0.95,"3,763,000","3,810,000",2024
230,Worry Be Gone Cottage & Carriage House,True,6,16,4.93,0.83,"279,000","286,000",2024


In [5]:
# This didn't matter in excel, but I now want the audience scores and critic scores to be 'scaled up' such that the a unit is 1 point
df['VRBO_Rating'] = df['VRBO_Rating'] * 100
df

,Property_Name,Is_Carriage_House,Bedrooms,Occupancy,Miles_to_Beach,VRBO_Rating,Zestimate_at_time_of_listing,Sale_Price,Year
0,114 North Winston Villa,False,5,14,0.10,100.0,"2,974,000","3,043,000",2023
1,151 Coopersmith Cottage & Carriage House,True,5,12,3.12,88.0,"1,554,000","1,453,000",2024
2,201 Rosemary Beach Inn,True,1,2,2.92,93.0,"2,386,000","2,433,000",2024
3,202 Rosemary Beach Inn,True,1,2,1.93,86.0,"4,357,000","4,339,000",2024
4,203 Rosemary Beach Inn,True,1,2,4.30,97.0,"1,860,000","1,926,000",2024
...,...,...,...,...,...,...,...,...,...
227,White Dove,False,5,14,1.66,96.0,"3,033,000","3,003,000",2023
228,Wiggle Room,False,2,4,1.04,87.0,"5,677,000","5,648,000",2023
229,Winston Manor,False,6,14,0.23,95.0,"3,763,000","3,810,000",2024
230,Worry Be Gone Cottage & Carriage House,True,6,16,4.93,83.0,"279,000","286,000",2024


In [6]:
# Convert object columns to strings
df['Zestimate_at_time_of_listing'] = df['Zestimate_at_time_of_listing'].str.replace(',', '').str.strip()
df['Sale_Price'] = df['Sale_Price'].str.replace(',', '').str.strip()

# Convert the cleaned strings to numeric values (floats or ints)
df['Zestimate_at_time_of_listing'] = pd.to_numeric(df['Zestimate_at_time_of_listing'], errors='coerce')
df['Sale_Price'] = pd.to_numeric(df['Sale_Price'], errors='coerce')

In [7]:
print(df.dtypes)
df

Property_Name                    object
Is_Carriage_House                  bool
Bedrooms                          int64
Occupancy                         int64
Miles_to_Beach                  float64
VRBO_Rating                     float64
Zestimate_at_time_of_listing      int64
Sale_Price                        int64
Year                              int64
dtype: object


,Property_Name,Is_Carriage_House,Bedrooms,Occupancy,Miles_to_Beach,VRBO_Rating,Zestimate_at_time_of_listing,Sale_Price,Year
0,114 North Winston Villa,False,5,14,0.10,100.0,2974000,3043000,2023
1,151 Coopersmith Cottage & Carriage House,True,5,12,3.12,88.0,1554000,1453000,2024
2,201 Rosemary Beach Inn,True,1,2,2.92,93.0,2386000,2433000,2024
3,202 Rosemary Beach Inn,True,1,2,1.93,86.0,4357000,4339000,2024
4,203 Rosemary Beach Inn,True,1,2,4.30,97.0,1860000,1926000,2024
...,...,...,...,...,...,...,...,...,...
227,White Dove,False,5,14,1.66,96.0,3033000,3003000,2023
228,Wiggle Room,False,2,4,1.04,87.0,5677000,5648000,2023
229,Winston Manor,False,6,14,0.23,95.0,3763000,3810000,2024
230,Worry Be Gone Cottage & Carriage House,True,6,16,4.93,83.0,279000,286000,2024


In [8]:
#Here I realized I need to scale for millions
df['Zestimate_at_time_of_listing'] = df['Zestimate_at_time_of_listing'] / 1_000_000
df['Sale_Price'] = df['Sale_Price'] / 1_000_000

# Prepare the features (X) and target variable (y)
features = df[['Is_Carriage_House', 'Bedrooms',  'Occupancy', 'Miles_to_Beach', 
               'VRBO_Rating', 'Zestimate_at_time_of_listing', 'Year']]
target_variable = df[['Sale_Price']]
target_variable

,Sale_Price
0,3.043
1,1.453
2,2.433
3,4.339
4,1.926
...,...
227,3.003
228,5.648
229,3.810
230,0.286


In [9]:
# Define numeric and categorical columns
numeric_features = ['Bedrooms',  'Occupancy', 'Miles_to_Beach',  'VRBO_Rating', 'Zestimate_at_time_of_listing', 'Year']
categorical_features = ['Is_Carriage_House']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse=False), categorical_features)
    ])

from sklearn.ensemble import RandomForestRegressor

model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_variable, test_size=0.2, random_state=42)

In [10]:
# Fit the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

#RMSE or root mean squared
rmse = np.sqrt(mse)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error (RMSE):", rmse, " Million")
print("R-squared Score:", r2)

/Users/kevinbradley/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Mean Squared Error: 0.07009179345957485
Root Mean Squared Error (RMSE): 0.26474854760616695  Million
R-squared Score: 0.9718413447732902


In [11]:
# Get feature names
cat_feature_names = model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
feature_names = numeric_features + cat_feature_names.tolist()

# Get coefficients
feature_importances = model.named_steps['regressor'].feature_importances_

# Create a dataframe of features and their coefficients
coef_df = pd.DataFrame({'Feature': feature_names, 'feature_importances': feature_importances})
coef_df = coef_df.sort_values('feature_importances', key=abs, ascending=False)

print("Feature Coefficients:")
print(coef_df)

Feature Coefficients:
                        Feature  feature_importances
4  Zestimate_at_time_of_listing             0.986409
1                     Occupancy             0.005081
3                   VRBO_Rating             0.003226
2                Miles_to_Beach             0.003011
5                          Year             0.001022
0                      Bedrooms             0.000954
6        Is_Carriage_House_True             0.000298


In [12]:
# Function to make predictions
def predict_sale(Zestimate_at_time_of_listing, Miles_to_Beach, Year, Occupancy, VRBO_rating, Bedrooms, Is_Carriage_House):
    new_data = pd.DataFrame({
        'Zestimate_at_time_of_listing': [Zestimate_at_time_of_listing],
        'Miles_to_Beach': [Miles_to_Beach], 
        'Year' :[Year] ,  
        'Occupancy' : [Occupancy], 
        'VRBO_Rating' : [VRBO_rating], 
        'Bedrooms' : [Bedrooms], 
        'Is_Carriage_House' : [Is_Carriage_House]
    })
    return model.predict(new_data)[0]

In [13]:
prediction = predict_sale(Zestimate_at_time_of_listing = 2, Miles_to_Beach = 4,
                                  Year = 2024, Occupancy = 6, VRBO_rating = 90, Bedrooms= 2, Is_Carriage_House = False)

print("Prediction for Sale:", prediction)

Prediction for Sale: 2.100859999999999
